In [ ]:
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import warnings

In [35]:
DJA_Catalog_DataFrame= pd.read_csv('./DJACatalog.csv')
DJA_File_Path_str=os.path.expanduser('~/DJAData/')

if os.path.exists(DJA_File_Path_str):
    DJA_File_list=np.array(os.listdir(DJA_File_Path_str))
print('Number of files in DJA directory:', len(DJA_File_list))

Number of files in DJA directory: 18507


In [ ]:
Sample_Indices_list=list()

for Index_In_DJA_Catalog_int in range(len(DJA_Catalog_DataFrame)):
    DJA_Catalog_row=DJA_Catalog_DataFrame.iloc[Index_In_DJA_Catalog_int]

    redshift_float=DJA_Catalog_row['z']

    if (redshift_float < 4) or (redshift_float > 6.5):
        continue

    Spectra_NGrating_int=DJA_Catalog_row.nGr

    if Spectra_NGrating_int <2:
        continue

    Sample_Indices_list.append(Index_In_DJA_Catalog_int)

print('Number of samples in DJA catalog:', len(Sample_Indices_list))
Sample_Indices_list= np.array(Sample_Indices_list)

In [ ]:
DJA_Root_Path = 'https://s3.amazonaws.com/msaexp-nirspec/extractions/'


for DJA_Catalog_Dataframe_Index_int in tqdm(Sample_Indices_list, desc='Processing DJA Catalog'):
    Object_Catalog_Dataframe_row = DJA_Catalog_DataFrame.iloc[DJA_Catalog_Dataframe_Index_int]

    Object_Prism_fits_File_Name_str = Object_Catalog_Dataframe_row.file
    Object_NGrating_int= Object_Catalog_Dataframe_row.nGr
    Object_Redshift_float = Object_Catalog_Dataframe_row.z
    Object_Grating_List_str= Object_Catalog_Dataframe_row.Grating
    Object_File_Root_Name_str = Object_Catalog_Dataframe_row.root

    for Grating_Type_str in Object_Grating_List_str.split('   '):
        Grating_Type_str = Grating_Type_str.strip().lower()

        Disperser_Name_str = Grating_Type_str[1:-1]

        Dual_Filter_bool = False

        if Disperser_Name_str == '140':
            Dual_Filter_bool = True
            Disperser_Filter_070_str=Grating_Type_str+'-f070lp'
            Disperser_Filter_100_str=Grating_Type_str+'-f100lp'

        elif Disperser_Name_str == '235':
            Dual_Filter_bool = False
            Disperser_Filter_Single_str=Grating_Type_str+'-f170lp'
        elif Disperser_Name_str == '395':
            Dual_Filter_bool = False
            Disperser_Filter_Single_str=Grating_Type_str+'-f290lp'

        if Dual_Filter_bool:
            continue
            # try:
            #     Grating_070_fits_File_Name_str = Object_Prism_fits_File_Name_str.replace('prism-clear', Disperser_Filter_070_str)
            #     Grating_100_fits_File_Name_str = Object_Prism_fits_File_Name_str.replace('prism-clear', Disperser_Filter_100_str)

            #     if not os.path.exists(os.path.join(DJA_File_Path_str, Grating_070_fits_File_Name_str)) and not os.path.exists(os.path.join(DJA_File_Path_str, Grating_100_fits_File_Name_str)):
            #         print('Downloading:', Grating_070_fits_File_Name_str)
            #         os.system(f'wget -P {DJA_Root_Path} {Grating_070_fits_File_Name_str}')
            #         os.system(f'wget -P {DJA_Root_Path} {Grating_100_fits_File_Name_str}')
            # except Exception as e:
            #     warnings.warn(f"Error processing {Object_Prism_fits_File_Name_str} for {Grating_Type_str}: {e}")
            #     continue

        else:
            try:
                Grating_Single_fits_File_Name_str = Object_Prism_fits_File_Name_str.replace('prism-clear', Disperser_Filter_Single_str)

                url= os.path.join(DJA_Root_Path,Object_File_Root_Name_str, Grating_Single_fits_File_Name_str)

                if not os.path.exists(os.path.join(DJA_File_Path_str, Grating_Single_fits_File_Name_str)):
                    print('Downloading:', Grating_Single_fits_File_Name_str)
                    os.system(f'wget -P {DJA_Root_Path} {url}')
            except Exception as e:
                warnings.warn(f"Error processing {Object_Prism_fits_File_Name_str} for {Grating_Type_str}: {e}")
                continue

